# Nearest stars from Gaia DR2

Notebook adapted from Jorge Villa's Python notebook

General packages to perform the Gaia query and write out the values

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table
from astroquery.gaia import Gaia
import os

Checking available tables in Gaia DR2

In [5]:
tables = Gaia.load_tables(only_names=True)
for table in (tables):
    print(table.get_qualified_name())

Retrieving tables...
Parsing tables...
Done.
public.public.dual
public.public.hipparcos
public.public.hipparcos_newreduction
public.public.hubble_sc
public.public.igsl_source
public.public.igsl_source_catalog_ids
public.public.tycho2
tap_schema.tap_schema.columns
tap_schema.tap_schema.key_columns
tap_schema.tap_schema.keys
tap_schema.tap_schema.schemas
tap_schema.tap_schema.tables
gaiadr1.gaiadr1.aux_qso_icrf2_match
gaiadr1.gaiadr1.ext_phot_zero_point
gaiadr1.gaiadr1.allwise_best_neighbour
gaiadr1.gaiadr1.allwise_neighbourhood
gaiadr1.gaiadr1.gsc23_best_neighbour
gaiadr1.gaiadr1.gsc23_neighbourhood
gaiadr1.gaiadr1.ppmxl_best_neighbour
gaiadr1.gaiadr1.ppmxl_neighbourhood
gaiadr1.gaiadr1.sdss_dr9_best_neighbour
gaiadr1.gaiadr1.sdss_dr9_neighbourhood
gaiadr1.gaiadr1.tmass_best_neighbour
gaiadr1.gaiadr1.tmass_neighbourhood
gaiadr1.gaiadr1.ucac4_best_neighbour
gaiadr1.gaiadr1.ucac4_neighbourhood
gaiadr1.gaiadr1.urat1_best_neighbour
gaiadr1.gaiadr1.urat1_neighbourhood
gaiadr1.gaiadr1.cepheid

Perform the Gaia DR2 query for all stars with parallax < 50 mas

In [8]:
Data_1 = Gaia.launch_job_async("SELECT gaia.source_id, gaia.ra, gaia.ra_error, gaia.dec, gaia.dec_error, gaia.parallax, gaia.parallax_error, gaia.l, gaia.b, gaia.phot_g_mean_mag+5*log10(gaia.parallax)-10.0 AS g_mag_abs, gaia.bp_rp, gaia.pmra, gaia.pmra_error, gaia.pmdec, gaia.pmdec_error, gaia.phot_bp_mean_mag, gaia.phot_rp_mean_mag, gaia.phot_g_mean_mag, gaia.astrometric_n_good_obs_al, gaia.astrometric_chi2_al, gaia.phot_bp_mean_flux_over_error, gaia.phot_rp_mean_flux_over_error, gaia.phot_bp_rp_excess_factor, gaia.visibility_periods_used, (gaia.astrometric_chi2_al/(gaia.astrometric_n_good_obs_al - 5.0)) AS unit_weight FROM gaiadr2.gaia_source AS gaia WHERE gaia.parallax >= 50.0 AND gaia.b >= -90.0 AND gaia.b <= 90.0 AND gaia.l >= 0.0 AND gaia.l <= 360.0 AND (gaia.parallax/gaia.parallax_error) > 10.0 AND gaia.phot_bp_mean_flux_over_error > 10 AND gaia.phot_rp_mean_flux_over_error > 10 AND gaia.phot_bp_rp_excess_factor > (1.0 + 0.015*power((gaia.phot_bp_mean_mag-gaia.phot_rp_mean_mag),2)) AND gaia.phot_bp_rp_excess_factor < (1.3 + 0.06*power((gaia.phot_bp_mean_mag-gaia.phot_rp_mean_mag),2))", dump_to_file = False)

Query finished.


Get the results and save the data to a CSV file

In [9]:
Data_1_DR2 = Data_1.get_results()

In [10]:
np.savetxt('./gaia_nearest_stars.csv', np.c_[Data_1_DR2["source_id"], Data_1_DR2["ra"], Data_1_DR2["ra_error"],\
                                Data_1_DR2["dec"], Data_1_DR2["dec_error"], Data_1_DR2["parallax"],\
                                Data_1_DR2["parallax_error"], Data_1_DR2["l"], Data_1_DR2["b"],\
                                Data_1_DR2["g_mag_abs"], Data_1_DR2["bp_rp"], Data_1_DR2["pmra"],\
                                Data_1_DR2["pmra_error"], Data_1_DR2["pmdec"], Data_1_DR2["pmdec_error"],\
                                Data_1_DR2["phot_bp_mean_mag"], Data_1_DR2["phot_rp_mean_mag"], Data_1_DR2["phot_g_mean_mag"],\
                                Data_1_DR2["astrometric_n_good_obs_al"], Data_1_DR2["astrometric_chi2_al"], Data_1_DR2["phot_bp_mean_flux_over_error"],\
                                Data_1_DR2["phot_rp_mean_flux_over_error"], Data_1_DR2["phot_bp_rp_excess_factor"],\
                                Data_1_DR2["visibility_periods_used"],\
                                Data_1_DR2["unit_weight"]], delimiter = ',',\
           header = 'source_id,ra,ra_error,dec,dec_error,parallax,parallax_error,l,b,g_mag_abs,bp_rp,pmra,pmra_error,pmdec,pmdec_error, phot_bp_mean_mag, phot_rp_mean_mag, phot_g_mean_mag, astrometric_n_good_obs_al,  astrometric_chi2_al, phot_bp_mean_flux_over_error, phot_rp_mean_flux_over_error, phot_bp_rp_excess_factor, vis_period, unit_weight',\
           fmt = ['%i','%.15f','%.15f','%.15f','%.15f',\
                  '%.15f','%.15f','%.15f','%.15f','%.15f',\
                  '%.15f','%.15f','%.15f','%.15f','%.15f',\
                  '%.15f','%.15f','%.15f','%.15f','%.15f','%.15f','%.15f','%.15f','%.15f','%.15f'])